# Algoritmos genéticos
--------------------

Tutorial completo: http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

Referencia DEAP: http://deap.readthedocs.io/en/master/api/tools.html

In [ ]:
import random
from deap import base, creator, tools

## Fitness

La clase "Fitness" proporcionada es una **clase abstracta** que necesita un atributo de pesos para ser funcional. Un "fitness" a minimizar se construye utilizando pesos negativos, mientras que para maximizar debemos coloar pesos positivos. Es posible que la función de fitness incluya varias funciones internas donde unas deban maximizarse y otras minimizarse. Por esta razón el parámetro "weights" es una tupla.

La función *create()* tiene al menos dos argumentos, un nombre para la clase recién creada y una clase base. Cualquier argumento subsiguiente se convierte en un **atributo de la clase**.

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
# creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))

## Individuos

El primer individuo que crearemos será una simple lista que contiene flotantes. Para producir este tipo de individuo, necesitamos crear una clase *Individual*, usando el creador, que heredará del tipo de lista estándar y tendrá un atributo fitness.

In [ ]:
creator.create("Individual", list, fitness=creator.FitnessMin)

Introducimos ahora el método *register( )*, que toma, al menos, dos argumentos; un alias y una función asignada a este alias. Cualquier otro argumento posterior se pasará a la función cuando esta se invoque. Por lo tanto, el código siguiente creará dos alias en el toolbox; *attr_float* e *individuo*. La primera redirecciona a la función *random.random( )*. El segundo es un acceso directo a la función *initRepeat( )*, fijando su argumento de contenedor a la clase *creator.Individual*, su argumento *func* a la función *toolbox.attr_float()* y su número de repeticiones al argumento *IND_SIZE*.

In [ ]:
IND_SIZE=10  # tamaño del individuo. 

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

Probemos algunas cosas...

In [ ]:
i = toolbox.individual()
print(len(i))
print(i)  # cada individuo es una lista de diez números flotantes.

In [ ]:
print(toolbox.attr_float())  # attr_float es un alias a la función random.random().

## Población

Nuestra población será una lista con tantos individuos como queramos establecer. Para ello, registraremos el método "population" como, de nuevo, un acceso directo a la función *tools.initRepeat*, donde llenaremos una lista "*list*"con individuos "*toolbox.individual*".

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Probemos esto...

In [ ]:
p = toolbox.population(3)
print("Número de individuos:", len(p))
print(p)  # tenemos 3 individuos con 10 flotantes cada uno.

# Podríamos haber registrado "population" también con un número fijado en la población.
# toolbox.register("population", tools.initRepeat, list, toolbox.individual, 3)

## Operadores

Los operadores son como los inicializadores, excepto que algunos ya están implementados en el módulo "toolbox". Una vez se hayan elegido los operadores adecuados, hay que registrarlos en "toolbox". La función (o funciones) de "fitness" nos las podemos crear nosotros.

In [ ]:
def evaluate(individual):  # Nuestra función de fitness será tan solo la suma de los 10 valores del individuo.
    return sum(individual),  # Fíjate en la coma al final, quiere decir que devolvemos una tupla.

toolbox.register("mate", tools.cxTwoPoint)  # Cruce en dos puntos
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

## Algoritmo final

In [ ]:
def main(pop):
    
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = [toolbox.evaluate(i) for i in pop]
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = [toolbox.clone(i) for i in offspring]

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = [toolbox.evaluate(i) for i in invalid_ind] 
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop = offspring

    return pop

In [ ]:
# Comprobemos la población al inicio 
pop = toolbox.population(n=50)

for k, i in enumerate(pop):
    total = 0
    for j in i:
        total += j
    print("Ind:",k, "->", total)
    
result = main(pop)

print("-------------------------------")
for k, i in enumerate(result):
    total = 0
    for j in i:
        total += j
    print("Ind:",k, "->", total)

## Ejercicios

1. Incrementa el número de generaciones y observa cómo cambia el resultado final.
2. Cambia el código para que cada individuo solo tenga 5 flotantes y maximice la suma total de cada individuo.
3. Crea un nuevo algoritmo genético que resuelva el [problema del viajante (TSP)](https://es.wikipedia.org/wiki/Problema_del_viajante). 

Consejos para implementar el TSP:
- Para crear las localizaciones de las ciudades (x,y) puedes hacer uso de números complejos, que ya Python incorpora por defecto. El valor absoluto de la resta de dos ciudades te devuelve la distancia entre ellas.
- Para crear el índice de las ciudades tienes [numpy.random.permutation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.permutation.html).
- El cruce de dos individuos los puedes hacer mediante la función [deap.tools.cxOrdered](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOrdered).


